# Pandas tip #1: Faster DataFrame row Iteration
Vectorizing the data is often the fastest way to process the data. There are however situations that you are still need to iterate through each row of a DataFrame. Reasons are most often that you solve a problem (too) "quick and dirty". This is fine for one-time processing and if it is in the order of minutes.

When iterating over each row, many use the .iterrows() function of the DataFrame. This method returns the index and a Series object. Pandas cannot guarantee that it returns a view or a copy, therefore, do not change the data you are iterating over directly. Changes you make might not be preserved. 

Pandas offers another method to iterate over each row: .itertuples(). The .itertuples() method returns a namedtuple object and depending on an option also returns the index. This method returns the same information as .iterrows() while not giving the illusion that you can change the original DataFrame using the iteration. But the best part is that it is 14 times faster than .iterrows(). Lets demonstrate this:

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame(
    np.random.randint(0, 100, size=(100, 4)),
    columns=list('ABCD'),
)

In [2]:
df

,A,B,C,D
0,71,3,89,90
1,15,60,54,34
2,98,78,95,75
3,45,16,43,29
4,55,17,17,78
...,...,...,...,...
95,28,86,49,52
96,90,94,63,40
97,83,92,79,42
98,64,63,30,1


Now lets use the `%%timeit` magic command from Jupyter to assess the speed of the iteration:

In [3]:
%%timeit

for ix, row in df.iterrows():
    pass

4.01 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
(ix, row) = next(df.iterrows())
print('row.A = ', row.A)
print('\nFull row Series object:')
print(row)

row.A =  71

Full row Series object:
A    71
B     3
C    89
D    90
Name: 0, dtype: int64


It takes about 3.9ms to iterate through the 100 rows, each having 4 columns.

Doing the same with .itertuples() is much faster, while having the same information:

In [5]:
%%timeit

for nt in df.itertuples():
    pass

299 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


The method is not only faster but also has the exact same information. To show this we need to get a new row first as the for loop has used all instances of the generator. In Python, when creating a generator, the items are cosumed and when the loop is finished, there are no items left.

In [6]:
nt = next(df.itertuples())
print('nt.A = ', nt.A)
print('\nFull nametuple object:')
print(nt)

nt.A =  71

Full nametuple object:
Pandas(Index=0, A=71, B=3, C=89, D=90)


The index gets the name 'index' and is accessible through `nt.index`. There is a tiny speed-up when you tell Pandas to exclude the index. However, for debugging, it is often easier to keep the index. The index can be excluded using:

In [7]:
%%timeit

for row in df.itertuples(index=False):
    pass

284 µs ± 7.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This is a simple method to increase the speed of your iterations. Of course, when writing code that is used often, invest the time of vectorizing you steps. If it is just short and simple one-time event, use the fastest iteration there is: .itertuples().

If you have any questions, comments, or requests, feel free to [contact me on LinkedIn](https://linkedin.com/in/dennisbakhuis).

In [9]:
%%timeit

for row in df[['A', 'C']].itertuples(index=False):
    pass

694 µs ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit

for A, C in zip(df['A'], df['C']):
    pass

19.6 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%%timeit

for A, B, C, D in zip(df['A'], df['B'],df['C'], df['D']):
    pass

37.4 µs ± 799 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
%%timeit

for A, B, C, D in df.values:
    pass

76 µs ± 252 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
%%timeit

_ = df.apply(lambda x: True)

459 µs ± 6.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
